In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
from IPython.display import clear_output

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',100)

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [3]:
def algo_test(x,y):

        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        XGBC=XGBRegressor()
        dt=DecisionTreeRegressor()
        kn=KNeighborsRegressor()
        
        
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=13)
        
        algos=[L,R,Lass,E,ETR,GBR,XGBC,dt,kn]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','XGradientBoosting','DecisionTree','KNeighbors']
        
        r2Score = []
        rmse = []
        mae = []
        
        result=pd.DataFrame(columns=['R2_score','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r2Score.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
            
        result['R2_score']=r2Score 
        result.RMSE=rmse
        result.MAE=mae


        return result.sort_values('R2_score',ascending=False)

In [2]:
tumiller = ['istanbul']

In [3]:
iller = []
sayfalar = []
for il in tumiller:
    url="https://www.zingat.com/" + il +"-kiralik-daire?page="
    iller.append(url)
z = 1
for i in range(0,len(iller)):
    clear_output(wait=True)
    print(z)
    illistesi = requests.get(iller[i]).text
    soup = bs(illistesi, 'html.parser')
    try:
        pag = soup.find("nav", "zng-pagination")['data-total']
        for j in range(2, int(pag)):
            sayfa = iller[i] + str(j)
            sayfalar.append(sayfa)
            z = z + 1
    except:
        continue
sayfalar.extend(iller)

1


In [4]:
sayfaurl = []
satisurl = []
k = 1
m = 1
for sayfa in sayfalar:
    clear_output(wait=True)
    print(m)
    req = requests.get(sayfa)
    soup = bs(req.text, "html.parser")
    sayfaurl.append(soup.select(".zl-card-inner"))
    m = m + 1
for page in sayfaurl:
    for link in page:
        satisurl.append("https://www.zingat.com" + link.get("href"))

47


In [5]:
ozellikler = []
fiyatlar = []
mahalle = []
ilce =[]
il = []
gun = []
ay = []
yil = []
n = 1

for url in satisurl:
    try:
        clear_output(wait=True) 
        print(n)
        req = requests.get(url)
        soup = bs(req.text, "html.parser")
        ozellikler.append([tag.text.strip() for tag in soup.find_all(["strong", "span"], {"class": "col-md-6"})])
        fiyatlar.append(soup.find("strong", itemprop = "price").text.replace("TL", "").replace(".","").replace(" ",""))
        mahalle.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[0])
        ilce.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[1])
        il.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[2])
        gun.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[0])
        ay.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[1])
        yil.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[2])
        n = n + 1
    except:
        continue
    
ozelliklerYeni = []
for ozellik in ozellikler:
    ozelliklerYeni.append(dict(zip(ozellik[::2], ozellik[1::2])))

987


In [8]:
df = pd.DataFrame(ozelliklerYeni)
df['Fiyat'] = pd.Series(fiyatlar)
df['İl'] = pd.Series(il)
df['İlçe'] = pd.Series(ilce)
df['Mahalle'] = pd.Series(mahalle)
df['İlan Gün'] = pd.Series(gun)
df['İlan Ay'] = pd.Series(ay)
df["İlan Yıl"] = pd.Series(yil)

df.drop(['Takasa Uygun','Aidat','Görüntülü Ev Gezintisi\n                                                                ?','Yapının Durumu','Ada','Kimden','Konut Şekli','Hayvan Dostu Patili Evler','Tapu Durumu','Parsel'], axis=1, inplace=True)

df = df['Oda Sayısı'].str.rsplit('+', 0, expand=True).join(df)
df=df.rename(columns = {0:'İlan Oda Sayısı'})
df=df.rename(columns = {1:'İlan Salon Sayısı'})
del df['Oda Sayısı']
df[['Brüt m²']] = df[['Brüt m²']].replace('m²', '', regex=True)
df

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11404\2194887577.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df = df['Oda Sayısı'].str.rsplit('+', 0, expand=True).join(df)


,İlan Oda Sayısı,İlan Salon Sayısı,İlan Tarihi,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,...,Depozito,Krediye Uygun,Mobilya Durumu,Fiyat,İl,İlçe,Mahalle,İlan Gün,İlan Ay,İlan Yıl
0,4,2,9 Ağustos 2023,4812043,140,160,2,4,Kombi (Doğalgaz),4,...,30.000 TL,Hayır,NaN,29000,İstanbul,Kağıthane,Merkez,9,Ağustos,2023
1,3,1,4 Ağustos 2023,4844894,125,140,2,8,Merkezi Sistem,1,...,25.500 TL,Hayır,Eşyasız,24000,İstanbul,Şişli,Merkez,4,Ağustos,2023
2,2,1,29 Temmuz 2023,3393464,87,97,2,4,Yerden Isıtma,4,...,40.000 TL,Hayır,Eşyasız,40000,İstanbul,Sarıyer,Zekeriyaköy,29,Temmuz,2023
3,2,1,1 Ağustos 2023,4839277,75,85,1,7,Kalorifer (Doğalgaz),6,...,28.500 TL,Hayır,Eşyalı (Mobilyalı),16150,İstanbul,Kağıthane,Çağlayan,1,Ağustos,2023
4,2,1,1 Ağustos 2023,4839259,65,75,1,7,Kalorifer (Doğalgaz),NaN,...,24.000 TL,Hayır,NaN,13600,İstanbul,Kağıthane,Çağlayan,1,Ağustos,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,3,1,7 Ağustos 2023,4846167,90,110,1,5,Kombi (Doğalgaz),3,...,40.000 TL,Hayır,Eşyalı (Mobilyalı),40000,İstanbul,Sarıyer,Zekeriyaköy,7,Ağustos,2023
983,2,1,4 Ağustos 2023,4570426,80,90,1,5,Merkezi Sistem (Isı Payı Ölçer),Bahçe katı,...,27.000 TL,Hayır,Eşyalı (Mobilyalı),29000,İstanbul,Şişli,Teşvikiye,4,Ağustos,2023
984,3,1,15 Ağustos 2023,4855694,100,110,1,6,NaN,5,...,0 TL undefined,Hayır,Eşyasız,11500,İstanbul,Esenler,TurgutReis,15,Ağustos,2023
985,3,1,31 Temmuz 2023,4837199,105,125,1,7,Merkezi Sistem,4,...,0 TL undefined,Hayır,Eşyasız,15500,İstanbul,Bahçelievler,Bahçelievler,31,Temmuz,2023


In [12]:
df.drop(["İl"], axis=1, inplace=True)

In [13]:
df.sample(5)

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Bina Yaşı,Kullanım Durumu,Depozito,Krediye Uygun,Mobilya Durumu,Fiyat,İlçe,Mahalle,İlan Gün,İlan Ay,İlan Yıl
393,1,1,4859743,60,70,1,10-20 arası,Merkezi Sistem (Isı Payı Ölçer),6,6-10 arası,Boş,0 TL,Hayır,Eşyalı (Mobilyalı),20000,Bayrampaşa,Kocatepe,18,Ağustos,2023
250,2,1,4559051,60,75,1,4,Kombi (Doğalgaz),NaN,36-40 arası,Boş,NaN,Hayır,NaN,14900,Beyoğlu,KalyoncuKulluğu,21,Ağustos,2023
525,3,1,4853872,150,180,2,10-20 arası,Kombi (Doğalgaz),13,21-25 arası,Boş,0 TL,Hayır,Eşyalı (Mobilyalı),24000,Beylikdüzü,AdnanKahveci,15,Ağustos,2023
350,2,1,4860531,70,75,1,6,NaN,3,3,Boş,0 TL,Hayır,Eşyasız,10000,Esenyurt,Saadetdere,19,Ağustos,2023
757,2,1,4851234,80,90,2,4,Kombi (Doğalgaz),1,6-10 arası,Boş,0 TL,Hayır,Eşyalı (Mobilyalı),14000,Beylikdüzü,Gürpınar,11,Ağustos,2023


In [14]:
df.to_csv('istzingatyedek.csv', index=False)

In [5]:
df=pd.read_csv("istzingatyedek.csv")

In [15]:
df["Isıtma Tipi"].value_counts()

Kombi (Doğalgaz)                     323
Merkezi Sistem (Isı Payı Ölçer)      153
Klima                                133
Merkezi Sistem                        84
Kalorifer (Doğalgaz)                  29
Yerden Isıtma                         25
Fancoil                                8
Yok                                    3
Kombi (Elektrikli)                     3
Combi Boiler (Natural Gas)             2
Soba (Doğalgaz)                        2
Kat Kaloriferi                         2
Soba (Kömür)                           2
Central System (Heat Share Meter)      1
Name: Isıtma Tipi, dtype: int64

In [16]:
df["Isıtma Tipi"] = df["Isıtma Tipi"].fillna('0')
df.loc[df["Isıtma Tipi"] == "Yok" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "None" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "Soba (Doğalgaz)" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "Soba (Kömür)" , "Isıtma Tipi"] = "0"

df.loc[df["Isıtma Tipi"] == "Kat Kaloriferi" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Merkezi Sistem (Isı Payı Ölçer)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kombi (Doğalgaz)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Doğalgaz)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kombi (Elektrikli)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Akaryakıt)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Kömür)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "-" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Merkezi Sistem" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central System" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central Heating (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central System (Heat Share Meter)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Fancoil" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Floor Heating" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Independent Boiler" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Combi Boiler (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Stove (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Stove (Coal)" , "Isıtma Tipi"] = "1"

df.loc[df["Isıtma Tipi"] == "Klima" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Air Conditioner" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Yerden Isıtma" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Jeotermal" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Geothermal" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Güneş Enerjisi" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Solar Power" , "Isıtma Tipi"] = "2"

In [17]:
df["Isıtma Tipi"].value_counts()

1    605
0    224
2    158
Name: Isıtma Tipi, dtype: int64

In [320]:
df = df[df['İlan no'].notna()]
df = df[df['Fiyat'].notna()]
#df = df[df['İlan Gün'].notna()]
#df = df[df['İlan Ay'].notna()]
#df = df[df['İlan Yıl'].notna()]
df = df[df['Bulunduğu Kat'].notna()]
#df = df[df['Bina Yaşı'].notna()]
df = df[df['İlan Oda Sayısı'].notna()]
df = df[df['İlan Salon Sayısı'].notna()]
df = df[df['Binadaki Kat Sayısı'].notna()]

df['Brüt m²'] = pd.to_numeric(df['Brüt m²'])
df.drop(df[(df['Brüt m²']>5000)].index, inplace=True)

In [321]:
df["İlan Salon Sayısı"].value_counts()

1             17138
2               448
0                64
0 (Stüdyo)       42
3                26
4                 3
Name: İlan Salon Sayısı, dtype: int64

In [322]:
df.loc[df["İlan Salon Sayısı"] == "0 (Stüdyo)" , "İlan Salon Sayısı"] = "0"

In [323]:
df["İlan Salon Sayısı"].value_counts()

1    17138
2      448
0      106
3       26
4        3
Name: İlan Salon Sayısı, dtype: int64

In [18]:
df["Krediye Uygun"].value_counts()

Hayır    987
Name: Krediye Uygun, dtype: int64

In [19]:
df.loc[df["Krediye Uygun"] == "Evet" , "Krediye Uygun"] = "1"
df.loc[df["Krediye Uygun"] == "Yes" , "Krediye Uygun"] = "1"
df.loc[df["Krediye Uygun"] == "Hayır" , "Krediye Uygun"] = "0"
df.loc[df["Krediye Uygun"] == "No" , "Krediye Uygun"] = "0"

In [20]:
df["Krediye Uygun"].value_counts()

0    987
Name: Krediye Uygun, dtype: int64

In [328]:
df.reset_index(drop=True, inplace=True)
df

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,Fiyat,İl,İlçe
0,1,1,4291369,60,70,1,5,1,3,1,3575000,İstanbul,Sarıyer
1,2,1,4307880,68,90,1,5,1,Bahçe katı,1,2499000,İstanbul,Beylikdüzü
2,1,1,4317006,55,70,1,10-20 arası,1,1,0,890000,İstanbul,Esenyurt
3,2,1,4316997,95,105,1,4,1,1,1,679000,İstanbul,Beylikdüzü
4,2,1,4316979,100,110,1,5,1,Yüksek Giriş,1,470000,İstanbul,Beylikdüzü
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,1,1,3921291,60,65,1,5,1,Çatı Katı,1,750000,Zonguldak,ZonguldakMerkez
17717,4,2,3719036,170,180,2,2,1,Kot 1,1,650000,Zonguldak,Alaplı
17718,2,1,4106599,80,90,1,5,1,4,0,1000000,Zonguldak,Alaplı
17719,3,2,4152965,220,300,2,10,1,10,1,1050000,Zonguldak,Alaplı


In [21]:
df[['Fiyat']] = df[['Fiyat']].replace('TRY','', regex=True)
df["Fiyat"]=df["Fiyat"].astype(float)
df

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Bina Yaşı,Kullanım Durumu,Depozito,Krediye Uygun,Mobilya Durumu,Fiyat,İlçe,Mahalle,İlan Gün,İlan Ay,İlan Yıl
0,4,2,4812043,140,160,2,4,1,4,11-15 arası,Boş,30.000 TL,0,NaN,29000.0,Kağıthane,Merkez,9,Ağustos,2023
1,3,1,4844894,125,140,2,8,1,1,40 ve üzeri,Boş,25.500 TL,0,Eşyasız,24000.0,Şişli,Merkez,4,Ağustos,2023
2,2,1,3393464,87,97,2,4,2,4,3,Boş,40.000 TL,0,Eşyasız,40000.0,Sarıyer,Zekeriyaköy,29,Temmuz,2023
3,2,1,4839277,75,85,1,7,1,6,11-15 arası,NaN,28.500 TL,0,Eşyalı (Mobilyalı),16150.0,Kağıthane,Çağlayan,1,Ağustos,2023
4,2,1,4839259,65,75,1,7,1,NaN,11-15 arası,Boş,24.000 TL,0,NaN,13600.0,Kağıthane,Çağlayan,1,Ağustos,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,3,1,4846167,90,110,1,5,1,3,21-25 arası,Mülk Sahibi,40.000 TL,0,Eşyalı (Mobilyalı),40000.0,Sarıyer,Zekeriyaköy,7,Ağustos,2023
983,2,1,4570426,80,90,1,5,1,Bahçe katı,26-30 arası,Boş,27.000 TL,0,Eşyalı (Mobilyalı),29000.0,Şişli,Teşvikiye,4,Ağustos,2023
984,3,1,4855694,100,110,1,6,0,5,26-30 arası,Boş,0 TL undefined,0,Eşyasız,11500.0,Esenler,TurgutReis,15,Ağustos,2023
985,3,1,4837199,105,125,1,7,1,4,26-30 arası,Boş,0 TL undefined,0,Eşyasız,15500.0,Bahçelievler,Bahçelievler,31,Temmuz,2023


In [22]:
old_cols = df.columns.values 
new_cols = ['İlan no','İlan Oda Sayısı', 'İlan Salon Sayısı',
       'Net m²', 'Brüt m²', 'Banyo Sayısı', 'Binadaki Kat Sayısı',
       'Isıtma Tipi', 'Bulunduğu Kat', 'Krediye Uygun',
       'İl', 'İlçe','Fiyat']
df = df.reindex(columns=new_cols)

In [23]:
df.to_csv("filtreVeriler.csv")

In [51]:
df

,İlan no,İlan Oda Sayısı,İlan Salon Sayısı,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,İlçe,Fiyat
0,4812043,4,2,140,160,2,4,1,4,0,Kağıthane,29000.0
1,4844894,3,1,125,140,2,8,1,1,0,Şişli,24000.0
2,3393464,2,1,87,97,2,4,2,4,0,Sarıyer,40000.0
3,4839277,2,1,75,85,1,7,1,6,0,Kağıthane,16150.0
4,4839259,2,1,65,75,1,7,1,2,0,Kağıthane,13600.0
...,...,...,...,...,...,...,...,...,...,...,...,...
982,4846167,3,1,90,110,1,5,1,3,0,Sarıyer,40000.0
983,4570426,2,1,80,90,1,5,1,0,0,Şişli,29000.0
984,4855694,3,1,100,110,1,6,0,5,0,Esenler,11500.0
985,4837199,3,1,105,125,1,7,1,4,0,Bahçelievler,15500.0


In [25]:
df.columns

Index(['İlan no', 'İlan Oda Sayısı', 'İlan Salon Sayısı', 'Net m²', 'Brüt m²',
       'Banyo Sayısı', 'Binadaki Kat Sayısı', 'Isıtma Tipi', 'Bulunduğu Kat',
       'Krediye Uygun', 'İl', 'İlçe', 'Fiyat'],
      dtype='object')

In [28]:
df.drop(["İl"], axis=1, inplace=True)

In [33]:
df["Bulunduğu Kat"].value_counts()

2     133
3     113
4     106
0      99
1      94
5      61
1      44
7      41
6      38
9      34
21     31
8      25
13     19
11     15
5      13
12     12
10     10
15      9
14      6
16      4
2       4
3       2
18      2
4       2
17      1
10      1
19      1
Name: Bulunduğu Kat, dtype: int64

In [32]:
df["Bulunduğu Kat"].replace({"Bahçe katı":0,"Giriş Katı":0,"Yüksek Giriş":1,"20 ve üzeri":21,"Çatı Katı":5,"En Üst Kat":5,"Kot 2":2,"Kot 4":4,"Müstakil":1,"Zemin Kat":0,"Kot 3":3,"Bodrum Kat":0,"Garden Floor":0,"Teras Kat":10,"Kot 1":1},regex=True,inplace=True)

In [ ]:
df.loc[df.dairenin_katı == "Yüksek Giriş", "dairenin_katı"] = "Giriş"
df.loc[df.dairenin_katı == "Düz Giriş", "dairenin_katı"] = "Giriş"
df.loc[df.dairenin_katı == "Yüksek Bodrum", "dairenin_katı"] = "Bodrum"
df.loc[df.dairenin_katı == "Tam Bodrum", "dairenin_katı"] = "Bodrum"
df.loc[df.dairenin_katı == "Yarı Bodrum", "dairenin_katı"] = "Bodrum"
df.loc[df.dairenin_katı == "10", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "11", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "12", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "13", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "14", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "15", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "16", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "17", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "18", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "19", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "20", "dairenin_katı"] = "10-20"
df.loc[df.dairenin_katı == "21", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "22", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "23", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "24", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "25", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "26", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "27", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "28", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "29", "dairenin_katı"] = "21-29"
df.loc[df.dairenin_katı == "Kot 1" , "dairenin_katı"] = "-1" 
df.loc[df.dairenin_katı == "Kot 2" , "dairenin_katı"] = "-2" 
df.loc[df.dairenin_katı == "Kot 3" , "dairenin_katı"] = "-3" 
df['dairenin_katı'] = df['dairenin_katı'].astype(str)

In [50]:
df.isnull().sum()

İlan no                0
İlan Oda Sayısı        0
İlan Salon Sayısı      0
Net m²                 0
Brüt m²                0
Banyo Sayısı           0
Binadaki Kat Sayısı    0
Isıtma Tipi            0
Bulunduğu Kat          0
Krediye Uygun          0
İlçe                   0
Fiyat                  0
dtype: int64

In [89]:
df['İlan Salon Sayısı'].value_counts()

1    940
2     30
0     17
Name: İlan Salon Sayısı, dtype: int64

In [104]:
df["Binadaki Kat Sayısı"].unique()

array(['4', '8', '7', '5', '6', 10, 20, '10', '2', '9', '3', '1'],
      dtype=object)

In [103]:
df['Binadaki Kat Sayısı'] = df['Binadaki Kat Sayısı'].replace("10-20 arası", 10)
df['Binadaki Kat Sayısı'] = df['Binadaki Kat Sayısı'].replace("20 ve üzeri", 20)

In [98]:
df['Banyo Sayısı'] = df['Banyo Sayısı'].replace("6 ve üzeri", 6)

In [87]:
#df['İlan Salon Sayısı'].replace({"0 (Stüdyo)":0}, regex=True,inplace=True)
df['İlan Salon Sayısı'] = df['İlan Salon Sayısı'].replace("0 (Stüdyo)", 0)

In [49]:
df['İlan Oda Sayısı'].fillna(df['İlan Oda Sayısı'].mean(), inplace=True)
df["İlan Salon Sayısı"].fillna(df["İlan Salon Sayısı"].mode()[0],inplace=True)
df['Banyo Sayısı'].fillna(2,inplace=True)
df['Binadaki Kat Sayısı'].fillna(df['Binadaki Kat Sayısı'].mode()[0],inplace=True)
df['Bulunduğu Kat'].fillna(df['Bulunduğu Kat'].mode()[0],inplace=True)

In [63]:
df['Fiyat'] = df['Fiyat'].replace({'[^\b\d+\b]':''}, regex=True).astype('float64')

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   İlan no              987 non-null    int32  
 1   İlan Oda Sayısı      987 non-null    int32  
 2   İlan Salon Sayısı    987 non-null    int32  
 3   Net m²               987 non-null    int32  
 4   Brüt m²              987 non-null    int32  
 5   Banyo Sayısı         987 non-null    int32  
 6   Binadaki Kat Sayısı  987 non-null    object 
 7   Isıtma Tipi          987 non-null    object 
 8   Bulunduğu Kat        987 non-null    object 
 9   Krediye Uygun        987 non-null    object 
 10  İlçe                 987 non-null    object 
 11  Fiyat                987 non-null    float64
dtypes: float64(1), int32(6), object(5)
memory usage: 69.5+ KB


In [105]:
for col in ('İlan no','İlan Oda Sayısı','İlan Salon Sayısı','Net m²','Brüt m²','Banyo Sayısı','Binadaki Kat Sayısı','Isıtma Tipi','Bulunduğu Kat','Krediye Uygun') : df[col]=df[col].astype(int)

In [68]:
dfd=pd.get_dummies(df,drop_first=True)

In [106]:
df_dummies = pd.get_dummies(df.drop(columns=['İlçe']), drop_first=True)
cols = df_dummies.columns.to_list()

In [107]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df_dummies=scaler.fit_transform(df_dummies)
df_dummies=pd.DataFrame(df_dummies, columns=cols)

In [108]:
x=df_dummies.drop(columns=['Fiyat'])
y=df_dummies['Fiyat']

In [69]:
from sklearn.model_selection import train_test_split
y=df[["Fiyat"]]
x=df.drop(columns='Fiyat',axis=1)

In [109]:
algo_test(x,y)

,R2_score,RMSE,MAE
Gradient Boosting,0.678793,0.043358,0.023017
DecisionTree,0.630429,0.046508,0.025620
Linear,0.589744,0.049001,0.028619
Extra Tree,0.576704,0.049774,0.029107
Ridge,0.571715,0.050067,0.028444
KNeighbors,0.538583,0.051967,0.024240
XGradientBoosting,0.435905,0.057459,0.026791
Lasso,-0.000919,0.076539,0.036768
ElasticNet,-0.000919,0.076539,0.036768
